In [ ]:
%load_ext autoreload
%autoreload 2

from cache import *
from costs import *
from features import *
from gradients import *
from helpers import *
from evaluate import *
from predict import *
from validate import *
from implementations import *
# from

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import csv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
SUB_SAMPLE = True
CACHE_DIR = "test/cache/" if SUB_SAMPLE else "cache/"
SUBMISSIONS_DIR = "test/submissions/" if SUB_SAMPLE else "submissions/"

y, x, ids = load_csv_data('data/train.csv', SUB_SAMPLE)

# 1 - Analytical Results

### Ridge Regression with Fixed Degree

##### Without Validation

In [ ]:
def clean_standardize_expand(y, x, h):
        
    degree = int(h['degree'])

    x = remove_errors(x)
    x = remove_outliers(x)
    x = standardize_all(x)
    x = remove_nan_features(x)
    x = build_poly(x, degree)

    return y, x

def ridge_regression_analytical(y, x, h):

    lambda_ = float(h['lambda'])
    degree = int(h['degree'])

    w = ridge_regression(y, x, lambda_)
    
    return {
        'w': w,
        'mse': compute_mse(y, x, w)
    }

def ridge_mse(y, x, w, h):

    lambda_ = float(h['lambda'])
    
    mse = compute_mse(y, x, w)
    ridge_norm = np.linalg.norm(w, 2) * lambda_
        
    return {
        'mse': mse,
        'ridge_norm': ridge_norm,
        'total_loss': mse + ridge_norm,
        'n_err': compute_error_count(predict_values)(y, x, w)
    }
    

In [ ]:
hs = { 
    'degree': [5, 6, 7], 
    'lambda': 1e-4,
}

_ = evaluate(
    clean = clean_standardize_expand, 
    fit   = ridge_regression_analytical, 
    x     = x, 
    y     = y, 
    hs    = hs, 
    cache = CACHE_DIR + 'clean_standardize_expand_ridge_regression_analytical'
)

##### Using Cross-Validation

Here, we implement the same model with cross-validation.

In [ ]:
hs = { 
    'degree': np.arange(4, 16), 
    'lambda': np.logspace(-8, -4, 5),
    'k_fold': 4,
    'seed': 0
}

def mse(y, x, w):
    return { 'mse' : compute_mse(y, x, w) }



evaluate(
    clean = cross_validate(ridge_regression_analytical, mse), 
    fit   = fit_function, 
    x     = x,
    y     = y, 
    hs    = hs, 
    cache = CACHE_DIR + 'clean_standardize_expand_cross_validate_ridge_regression_analytical_mse'
)

In [ ]:
# myModel.predict(best_h, x, y, SUBMISSIONS_DIR + 'RidgeRegression_MSE_FixedDegree_CrossValidation_Model')

## Gradient Descents

#### Least Square

#### Ridge Regression

#### Lasso

# Logistic Regression

In [ ]:
def map_logistic(clean):
    
    def inner_function(y, x, h):
        y, x = clean(y, x, h)
        y = np.where(y == 1, 1, 0)
        return y, x
    
    return inner_function

def logistic_gradient(y, x, w, h):
    
    return compute_logistic_gradient(y, x, w)
            


def logistic_gradient_ridge(y, x, w, h):
    
    lambda_ = h['lambda']
    
    return compute_logistic_gradient(y, x, w) + lambda_ * w



def logistic_gradient_lasso(y, x, w, h):
    
    lambda_ = h['lambda']
    
    return compute_logistic_gradient(y, x, w) + lambda_ * np.sign(w)



### Without Validation

##### Stochastic Gradient Descent

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': np.concatenate([[-2], np.arange(1, 7)]),
    'gamma': [1e-2, 1e-3], 
    'k_fold': 4,
    'lambda': 0,
    'max_iters': 3000,
    'num_batches': 1,
    'seed': 1,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_descent')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = descent_with_loss(stochastic_gradient_descent_e(logistic_gradient), logistic_error),  
        round_size = 100,
        cache      = cache,
        multiple   = False,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

###### Stochastic Gradient Descent With Ridge Regression

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': [-2] + np.arange(3, 4),
    'gamma': [1e-2, 1e-3], 
    'lambda': [1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 1,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_ridge_descent')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = stochastic_gradient_descent_e(logistic_gradient_ridge), 
        loss       = logistic_error_and_ridge, 
        round_size = 100,
        cache      = cache,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

##### Stochastic Gradient Descent With Lasso

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': [-2, 1, 2, 3, 4, 5, 6],
    'gamma': [1e-1, 1e-2, 1e-3], 
    'lambda': [1e-1, 1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 2000,
    'num_batches': 1,
    'seed': 0,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_lasso_descent')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = stochastic_gradient_descent_e(logistic_gradient_lasso), 
        loss       = logistic_error_and_lasso, 
        round_size = 100,
        cache      = cache,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

### With Cross-Validation

##### Stochastic Gradient Descent

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': np.concatenate([[-2], np.arange(1, 7)]),
    'gamma': [1e-2, 1e-3], 
    'k_fold': 4,
    'lambda': 0,
    'max_iters': 3000,
    'num_batches': 1,
    'seed': 0,
    'seed_cv': 0
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_descent_cross_validate')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = cross_validate_descent(
            stochastic_gradient_descent_e(logistic_gradient), 
            logistic_error
        ),
        round_size = 100,
        cache      = cache,
        multiple   = True,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

##### Descent with Ridge

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': [-2] + np.arange(3, 4),
    'gamma': [1e-2, 1e-3], 
    'lambda': [1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 1,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_ridge_descent_cross_validate')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = cross_validate_descent(
            stochastic_gradient_descent_e(logistic_gradient_ridge), 
            logistic_error_and_ridge,
        ),
        round_size = 100,
        cache      = cache,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

##### Descent with Lasso

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': [-2] + np.arange(3, 4),
    'gamma': [1e-2, 1e-3], 
    'lambda': [1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 1,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_lasso_descent_cross_validate')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = cross_validate_descent(
            stochastic_gradient_descent_e(logistic_gradient_lasso), 
            logistic_error_and_lasso,
        ),
        round_size = 100,
        cache      = cache,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

# Split Dataset

### Prepare Data

In [ ]:
y_split, x_split, _ = split_data(y, x, ids)

### Least Square Ridge

In [ ]:
hs = { 
    'degree': np.concatenate([np.array([-2]), np.arange(4, 16)]), 
    'lambda': np.logspace(-8, -4, 5),
    'k_fold': 4,
    'seed': 0
}

for i in range(5):
    
    cache = Cache(CACHE_DIR + f'clean_standardize_expand_cross_validate_ridge_regression_analytical_mse_split{i}')

    result = evaluate(
        clean = clean_standardize_expand, 
        fit   = fit_with_cache(cross_validate(ridge_regression_weights, mse_and_ridge), cache), 
        x     = x_split[i],
        y     = y_split[i], 
        hs    = hs
    )

In [ ]:
y_split, x_split, _ = split_data(y, x, ids)

# for i in range(5):
#     y_clean, x_clean = clean_standardize_expand(y_clean, x_clean, h)

h_array = [
    { 'degree': -2, 'lambda': 1e-8, 'k_fold': 4, 'seed': 0 },
    { 'degree': 12, 'lambda': 1e-7, 'k_fold': 4, 'seed': 0 },
    { 'degree': 11, 'lambda': 1e-4, 'k_fold': 4, 'seed': 0 },
    { 'degree': 11, 'lambda': 1e-8, 'k_fold': 4, 'seed': 0 },
    { 'degree': 10, 'lambda': 1e-8, 'k_fold': 4, 'seed': 0 }
]

caches = [Cache(CACHE_DIR + f'clean_standardize_expand_cross_validate_ridge_regression_analytical_mse_split{i}') for i in range(5)]


res = [
    evaluate(
        clean_and_fit_with_cache(
            clean_standardize_expand, 
            cross_validate(ridge_regression_weights, mse_and_ridge),
            caches[i]
        ),
        x     = x_split[i],
        y     = y_split[i], 
        hs    = h_array[i]
    )[0] for i in range(5)
]   

In [ ]:
res

In [ ]:
ws = [r['w'] for r in res]

In [ ]:
y_test, x_test, ids_test = load_csv_data('data/test.csv', SUB_SAMPLE)

In [ ]:
_, x_test_split, ids_test_split = split_data(y_test, x_test, ids_test)

In [ ]:
x_test_split_prep = [clean_standardize_expand(None, x_test_split[i], h_array[i])[1] for i in range(5)]

In [ ]:
y_pred = split_predict(predict_values, x_test_split_prep, ws, ids_test_split)

In [ ]:
create_csv_submission(ids_test, y_pred, SUBMISSIONS_DIR + 'clean_standardize_expand_cross_validate_ridge_regression_analytical_mse_split')

###  Logistic Ridge

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': np.concatenate([np.array([-2]), np.arange(1, 4)]), 
    'gamma': [1e-2, 1e-3], 
    'lambda': [1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 0,
    'seed_cv': 0
}

for i in range(5):
    
    cache = Cache(CACHE_DIR + f'clean_standardize_expand_logistic_ridge_regression_cross_validate_split{i}')

    _ = evaluate(
        clean = map_logistic(clean_standardize_expand), 
        fit   = descent_with_cache(
            descent    = cross_validate_descent(
                            stochastic_gradient_descent_e(logistic_gradient_ridge), 
                            logistic_error_and_ridge,
                        ),
            round_size = 100,
            cache      = cache,
            log        = True
        ), 
        y     = y_split[i],
        x     = x_split[i],
        hs    = hs
    )

### Logistic Lasso

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': np.concatenate([np.array([-2]), np.arange(1, 4)]), 
    'gamma': [1e-2, 1e-3], 
    'lambda': [1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 0,
    'seed_cv': 0
}

for i in range(5):
    
    cache = Cache(CACHE_DIR + f'clean_standardize_expand_logistic_lasso_regression_cross_validate_split{i}')

    _ = evaluate(
        clean = map_logistic(clean_standardize_expand), 
        fit   = descent_with_cache(
            descent    = cross_validate_descent(
                            stochastic_gradient_descent_e(logistic_gradient_lasso), 
                            logistic_error_and_lasso,
                        ),
            round_size = 100,
            cache      = cache,
            log        = True
        ), 
        y     = y_split[i],
        x     = x_split[i],
        hs    = hs
    )